In [1]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd
from datetime import datetime
from collections import Counter
import re

In [2]:
data = pd.read_csv(os.path.join(WDIR, 'true-north/true_north_clean.csv'))
data.drop(columns='associated Company Domain (Contact Level)', inplace=True)

In [3]:
summary = pd.DataFrame(columns=data.columns.to_list())
summary.loc['count'] = data.count()
mode = data.mode().head(1)
mode.reset_index(drop=True, inplace=True)
mode.rename(index={0: 'top'}, inplace=True)
summary = pd.concat([summary, mode])
freq = data.apply(lambda x: x.value_counts(dropna=True).max())
summary.loc['freq'] = freq

unique = data.nunique(dropna=True)
summary.loc['unique'] = unique

summary

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company Name,City,Which theme of the True North report do you most identify with and could support activity around?,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,sector,location,How would you like to be involved with the True North network?,company_size
count,245,245,185,184,183,183,181,21,185,0.0,0.0,241,109,50,109
top,2024-06-20 00:00:00,2024-06-20 00:00:00,Yes,Yes,No,No,Blackpool Council,Liverpool,"People, skills and the future",NaN,NaN,Education,Greater Manchester,Keeping informed about the latest True North n...,0-9
freq,60.0,60.0,179.0,182.0,160.0,97.0,2.0,5.0,56.0,NaN,NaN,8.0,28.0,18.0,35.0
unique,163,159,2,2,2,2,177,13,14,0.0,0.0,179,28,7,7


In [4]:
# summary = data.describe()
summary.loc['top_percent_of_count'] = (summary.loc['freq'] * 100 / summary.loc['count']).astype(float).round(1)

In [5]:
summary

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company Name,City,Which theme of the True North report do you most identify with and could support activity around?,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,sector,location,How would you like to be involved with the True North network?,company_size
count,245,245,185,184,183,183,181,21,185,0.0,0.0,241,109,50,109
top,2024-06-20 00:00:00,2024-06-20 00:00:00,Yes,Yes,No,No,Blackpool Council,Liverpool,"People, skills and the future",NaN,NaN,Education,Greater Manchester,Keeping informed about the latest True North n...,0-9
freq,60.0,60.0,179.0,182.0,160.0,97.0,2.0,5.0,56.0,NaN,NaN,8.0,28.0,18.0,35.0
unique,163,159,2,2,2,2,177,13,14,0.0,0.0,179,28,7,7
top_percent_of_count,24.5,24.5,96.8,98.9,87.4,53.0,1.1,23.8,30.3,NaN,NaN,3.3,25.7,36.0,32.1


In [6]:
company_names = pd.Series(data['Company Name'].sort_values())
print(company_names.unique())

['2030hub' 'AHR Architects' 'Abeceder ltd' 'Access Creative College'
 'Accountable Recruitment Limited'
 'Advanced Manufacturing Research Centre' 'Agent' 'Agile Automations'
 'Andrea Nixon Consulting' 'Armstrong Watson' 'Azets'
 'B engineering group' 'BHP LLP' 'BPP' 'Bank of England'
 'Barton Legal Limited' 'Big Hand 4 Business' 'Blackpool Council'
 'Business Doctors' 'Business Health Institute' 'Business of Science Ltd'
 'CBI' 'CH Marketing & Property' 'Cabron Happy World' 'Cazenove Capital'
 'Chris Shakespeare Coaching' 'Codurance Ltd'
 'Cortus Advisory Group Limited' 'DES Energy Ltd' 'DML Consulting'
 'Danaus Limited' 'Daneel Williams LLP' 'Deloitte LLP' 'E-Rail'
 'EMM European Movement Merseyside [LCR Liverpool City Region]' 'ESGmark'
 'Elevate' 'Endless' 'Eric Wright Civil Engineering Ltd.'
 'Essential Sourcing' 'FORT Search Limited' 'FSB'
 'Faith Brand Communications' 'Farrans Construction' 'Finance Yorkshire'
 'Fleetsolve Ltd' 'Flex Legal Ltd' 'Fox Brothers Group' 'Fragomen LLP'

There is LLoyds banking group and lloyds bank plc - we're assuming these are different companies.

In [7]:
# convert the month column to a datetime object
data['month'] = pd.to_datetime(data['Create Date'])

#convert the item to a formatted value in yyyy-mm format.
data['month_formatted'] = data['month'].apply(datetime.strftime, format='%Y-%m').sort_index(ascending=True)

In [8]:
# take the number of members and calculate the number that joined each month.
monthly_members = pd.DataFrame(data['month_formatted'].value_counts(ascending=False)).reset_index()
monthly_members['start_of_month'] = monthly_members['month_formatted'].astype(str) + '-01'

In [9]:
monthly_orgs = data.drop_duplicates(subset='Company Name', keep='last')
monthly_orgs = pd.DataFrame(monthly_orgs['month_formatted'].value_counts(ascending=False)).reset_index()
monthly_orgs['start_of_month'] = monthly_orgs['month_formatted'].astype(str) + '-01'

In [10]:
def decimal_date(data):
    # make a unix timestamp column
    data['timestamp'] = pd.to_datetime(data['month_formatted'], format='%Y-%m').astype(int) / 10**9
    # make a decimal date and round to 2dp.
    data['year'] = data['timestamp'].div((86400*365.25)).add(1970).round(2)
    # drop the timestamp column
    data.drop(columns='timestamp', inplace=True) 
    # set year and formatted month as the index so they aren't included in the cumsum.
    data.set_index(['year', 'month_formatted', 'start_of_month'], inplace=True, append=True)

    return data

In [11]:
def calculate_cumsum(data, count_name):
    # order by date, then do the cumsum. reset the index, drop the original index column as not needed
    data = pd.DataFrame(data.sort_index(level=2).cumsum(skipna=True).reset_index().drop(columns='level_0'))
    data.rename(columns={'count': f'{count_name}'}, inplace=True)
    return data

In [12]:
# apply above functions to data
cs_monthly_members = calculate_cumsum(decimal_date(monthly_members), count_name='individuals')
cs_monthly_orgs = calculate_cumsum(decimal_date(monthly_orgs), count_name='orgs')

In [13]:
# merged the two dataframes
cs_merged = cs_monthly_members.merge(cs_monthly_orgs, how='inner', on=['year', 'month_formatted', 'start_of_month'])

In [14]:
# write to file
cs_merged.to_csv(os.path.join(SRC_DIR,'overview/membership/_data/cumsum.csv'), index=False)

Calculating some summary stats

In [15]:
total_members = len(data.index)

total_companies = summary.loc['unique', 'Company Name']

top_company_size = summary.loc['top', 'company_size']

top_company_size_pct = summary.loc['top_percent_of_count', 'company_size']

# top_industry = summary.loc['top', 'Industry']
top_industry = ''
# top_industry_pct = summary.loc['top_percent_of_count', 'Industry']
top_industry_pct = ''

In [16]:
data['City']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
240    Blackburn
241      Chester
242       Helsby
243          NaN
244    Liverpool
Name: City, Length: 245, dtype: object

### Word frequency in the sector column

In [17]:
def normalize_string(s):
    # Convert to lowercase and remove non-alphanumeric characters (keeping spaces)
    try:
        s = s.lower()
    except:
        print(f"'{s}' is not a stirng type. Converting to string\n")
        return str(s)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    return s
data['normalized_sector'] = data['sector'].apply(normalize_string).str.split(';')
 
# Flatten the list and further split by spaces to handle multi-word strings
all_words = [word for sublist in data['normalized_sector'] for item in sublist for word in item.split()]
 
# Count the occurrences of each word
word_counts = Counter(all_words)
 
# Find the most common word
most_common_words = word_counts.most_common(10)
least_common_words = word_counts.most_common()

banned_words = ['and', 'or', 'of', 'it', 'the', 'for', 'with', 'we']
# print("The 10 most common words, not including 'and' are:") 
# for word, count in most_common_words:
#     if word == banned_words:
#         continue
#     print(f"'{word}' with {count} occurences")

print("The most common words, with at least 2 occurences are:")
words = []
counts = []
for word, count in least_common_words:
    if word in banned_words:
        continue
    if word == 'estate':
        continue
    if count >= 10:
        if word == 'real':
            words.append('real estate')
        else:
            words.append(word)
        counts.append(count)
        print(f"'{word}' with {count} occurences")

sector_counts = pd.DataFrame(data={'name': words, 'count': counts}).set_index('name')
# sector_counts['colour'] = round((sector_counts['count'] - min(sector_counts['count'])) / sector_counts['count'].max(), 3)
sector_counts.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/sector_word_counts.csv'))
# sector_strings = [item for sublist in data['sector'].str.split(';') for item in sublist]
# string_counts = Counter(sector_strings)
# print(string_counts.most_common())


'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

'nan' is not a stirng type. Converting to string

The most common words, with at least 2 occurences are:
'services' with 35 occurences
'construction' with 21 occurences
'education' with 20 occurences
'manufacturing' with 19 occurences
'professional' with 18 occurences
'business' with 17 occurences
'engineering' with 17 occurences
'technology' with 16 occurences
'finance' with 15 occurences
'development' with 14 occurences
'marketing' with 11 occurences
'real' with 11 occurences
'social' with 10 occurences
'recruitment' with 10 occurences


In [18]:
advisory_council = 11

if summary.loc['top', "Are you currently a B Corp or in the process of becoming a B Corp?"] == 'No':
    b_corps_pct = round(100 - summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"], 1)
else:
    summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"]

membership_increase = cs_merged['individuals'].pct_change().mul(100).iloc[-1].round(1)

northern_stars = len(pd.read_csv(os.path.join(SRC_DIR, 'overview/northern-stars/_data/northern_stars.csv')))

names = ["Total members", 
         "Total companies", 
         "Membership increase", 
         f"Companies with {top_company_size} employees", 
         "Top industry", 
         "Geographic reach", 
         "Advisory council", 
         "Northern stars", 
         "B Corps"]

values = [total_members, 
          total_companies, 
          membership_increase, 
          top_company_size_pct, 
          top_industry_pct, 
          "4", 
          advisory_council, 
          northern_stars, 
          b_corps_pct]

footnotes = ["People", 
             "Unique companies", 
             "Since last month", 
             "Of members work in companies of this size", 
             f"Of members work in {top_industry}", 
             "placeholder", 
             "Members represent the network on the True North advisory council", 
             "Companies have been featured as Northern Stars", 
             "Of member's organisations are B Corps or are joining"]

posts = ['','','%','%','%','','','','%']

urls = ['/overview/membership', '/overview/membership', '/overview/membership', '', '', '', 'https://www.brabners.com/insights/true-north/true-north-advisory-council-launches', '/overview/northern-stars', '/themes/sustainable-growth/b-corporations']
dashboard = pd.DataFrame(data={'name':names, 'value': values, 'footnote': footnotes, 'post': posts, 'url': urls})

dashboard

,name,value,footnote,post,url
0,Total members,245,People,,/overview/membership
1,Total companies,177,Unique companies,,/overview/membership
2,Membership increase,33.2,Since last month,%,/overview/membership
3,Companies with 0-9 employees,32.1,Of members work in companies of this size,%,
4,Top industry,,Of members work in,%,
5,Geographic reach,4,placeholder,,
6,Advisory council,11,Members represent the network on the True Nort...,,https://www.brabners.com/insights/true-north/t...
7,Northern stars,5,Companies have been featured as Northern Stars,,/overview/northern-stars
8,B Corps,12.6,Of member's organisations are B Corps or are j...,%,/themes/sustainable-growth/b-corporations


In [19]:
dashboard.to_csv(os.path.join(SRC_DIR, '_data/dashboard/true_north_members_list.csv'), index=False)